In [1]:
from OperacionesCSV import importar_CSV
from Neo4HungerGames import Neo4HungerGames
from Neo4Dataframes import Neo4Dataframes
import pandas as pd
from flask import Flask, render_template, redirect, url_for, session, request
import os
import uuid
from Controller import Controller

In [2]:
uri = "bolt://localhost:7687"
user = "mdad"
password = "123456789"

In [5]:
ruta_csv='HungerGames_Characters_Dataset_ALL.csv'

hgDF = importar_CSV(ruta_csv)
hgDF['ID'] = hgDF.index
dataConnection = Neo4Dataframes(uri, user, password)
dataConnection.create_nodes_links_from_df(hgDF)

--------------------------------------------


In [6]:
app = Flask(__name__)
neoConnection = Neo4HungerGames(uri, user, password)

# Ruta principal
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/characters', methods=['GET'])
def characters():
    districts = neoConnection.get_all_districts()
    return render_template('characters.html', districts=districts)

@app.route('/characters/all', methods=['GET'])
def all_characters():
    characters = neoConnection.get_all_characters()
    return render_template('characters_list.html', characters=characters)

@app.route('/characters/create', methods=['POST'])
def create_character():
    return Controller.create_character(neoConnection, request=request)

@app.route("/characters/link_game", methods=["GET", "POST"])
def link_character_game():
    return Controller.link_character_game(neoConnection, request=request)

@app.route("/characters/update/<int:character_id>", methods=["GET"])
def edit_character_form(character_id):
    character = neoConnection.get_character_by_id(character_id)
    return render_template("update_character.html", character=character, character_id=character_id)

@app.route("/characters/update/<int:character_id>", methods=["POST"])
def update_character(character_id):
    return Controller.update_character(neoConnection, request, character_id)

@app.route("/characters/delete", methods=["POST"])
def delete_character():
    return Controller.delete_character(neoConnection, request)

@app.route("/characters/delete/<character_id>", methods=["POST"])
def delete_character_by_id(character_id):
    return Controller.delete_character_by_id(neoConnection, character_id)



In [4]:
if __name__ == '__main__':
    app.run(debug=True, host="0.0.0.0", port=8080,use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.1.38:8080
Press CTRL+C to quit
127.0.0.1 - - [13/May/2025 20:17:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2025 20:17:09] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [13/May/2025 20:17:12] "GET /characters HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2025 20:17:12] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [13/May/2025 20:17:14] "GET /characters/all HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2025 20:17:14] "GET /static/list.css HTTP/1.1" 304 -
127.0.0.1 - - [13/May/2025 20:17:15] "GET /characters/delete/199 HTTP/1.1" 405 -
127.0.0.1 - - [13/May/2025 20:17:33] "GET /characters/delete/115 HTTP/1.1" 405 -
127.0.0.1 - - [13/May/2025 20:19:29] "GET /characters/all HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2025 20:19:29] "GET /static/list.css HTTP/1.1" 304 -
c:\Users\sarag\anaconda3\Lib\site-packages\neo4j\_sync\work\result.py:625: UserWarning: Expected a result with a single reco

In [4]:
neoConnection.delete_all_links_and_nodes()
neoConnection.close()